In [37]:
import pandas as pd
import numpy as np
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.mode.chained_assignment = None

Read data

In [38]:
df_races = pd.read_csv('01_raw/races.csv')
df_circuits = pd.read_csv('01_raw/circuits.csv')
df_drivers = pd.read_csv('01_raw/drivers.csv')
df_results = pd.read_csv('01_raw/results.csv')

In [39]:
df_results.head(1)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.000,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1


In [40]:
df_races.head(1)

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...


In [41]:
df_circuits.head(1)

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.850,144.968,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...


In [42]:
df_drivers.head(1)

,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton


Create master dataset

In [43]:
df_drivers = df_drivers[['driverId', 'driverRef', 'nationality']]
df_drivers.columns = ['driverId', 'driverRef', 'driver_nationality']
df = df_results.merge(df_drivers, on='driverId', how='left', validate='m:1')

In [44]:
df_races = df_races[['raceId', 'year', 'round', 'name', 'date', 'time', 'circuitId']]
df_races.columns = ['raceId', 'year', 'round', 'race_name', 'race_date', 'race_time', 'circuitId']
df = df.merge(df_races, on='raceId', how='left', validate='m:1')

In [45]:
df_circuits = df_circuits[['circuitId', 'name', 'country', 'alt']]
df_circuits.columns = ['circuitId', 'circuit_name', 'circuit_country', 'circuit_alt']
df = df.merge(df_circuits, on='circuitId', how='left', validate='m:1')